In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pprint
import psycopg2
import psycopg2.extras
import random

pp = pprint.PrettyPrinter(indent=4)

# Connecting to the DB

Set up the connection properties you need to access the database. Connect to the database using **.connect** with following connection parameters. Remember your username and password can be accessed directly from the **.pgpass** file.

In [ ]:
db_host = 'mlpolicylab.db.dssg.io'
db_database = "tech_session_nc"

In [ ]:
pgsql_connection = psycopg2.connect(host=db_host, database=db_database)
pp.pprint(pgsql_connection.get_dsn_parameters())

Use the connection's cursor to execute and run queries. The cursor can be used with different output formats: each row in the database can be represented in both list as well as dictionary datatypes.

In [ ]:
# As default, results are returned as a list of columns (similar to csv)
pgsql_cursor = pgsql_connection.cursor()

# We can also get results as a dictionary, where values are mapped to column names (preferred)
pgsql_cursor_dict = pgsql_connection.cursor(cursor_factory=psycopg2.extras.DictCursor)

# Executing Queries (using dicts)

Let's count the number of rows in one of our tables (inmt4aa1).

In [ ]:
# SQL query.
sql_count = 'SELECT COUNT(*) AS row_count FROM raw.inmt4aa1;'

# Execute it with dict cursor.
pgsql_cursor_dict.execute(sql_count)

# Fetch first (and only) row, then output the count
first_row = pgsql_cursor_dict.fetchone()

# Format of the first_row variable
print('First row datatype is {}'.format(type(first_row)))

# First row can be accessed as python dictionary.
# Notice the key is the same name as the AS clause above.
print('row_count = {}'.format(first_row['row_count']))

Now we'll retrieve rows from the table. However, we'll only fetch the first result.

In [ ]:
# SQL query.
sql_select = 'SELECT * FROM raw.inmt4aa1 LIMIT 10;'
pgsql_cursor_dict.execute(sql_select)
first_row = pgsql_cursor_dict.fetchone()

# Keys in first_row variable are the columns of raw.inmt4aa1 table
print("First Row Keys")
pp.pprint(list(first_row.keys()))
print("\n-----------------------------------------------------------------\n")

# Access values in the first_row varaible
print("First Row Values")
pp.pprint({k: v for k, v in first_row.items()})


Now, let's fetch multiple results

In [ ]:
sql_select = 'SELECT * FROM raw.inmt4aa1 LIMIT 10;'
pgsql_cursor_dict.execute(sql_select)

# First 10 rows.
result_list = pgsql_cursor_dict.fetchmany(size=10)

# Let's understand the datatype returned by fetchmany
print(type(result_list))
print(type(result_list[0]))

# loop
result_counter = 0
for i, result_row in enumerate(result_list):
    print('\n------------------------------------------------\n')
    print('Row {}'.format(i))
    pp.pprint({k: v for k, v in result_row.items()})

# Executing Queries (using lists)

Now let's see how things would change if we were using the default cursor.

In [ ]:
# SQL query.
sql_count = 'SELECT COUNT(*) as row_count FROM raw.inmt4aa1;'

# execute it with list cursor.
pgsql_cursor.execute(sql_count)
first_row = pgsql_cursor.fetchone()

print('First row is {}'.format(type(first_row)))

# First row should now be accessed as a list
print('row_count = {}'.format(first_row[0]))

In [ ]:
# SQL query.
sql_select = 'SELECT * FROM raw.inmt4aa1 LIMIT 10;'
pgsql_cursor.execute(sql_select)
first_row = pgsql_cursor.fetchone()
# Format of the first_row variable
print('First row datatype is {}'.format(type(first_row)))
print()

# Notice first_row is a list and thus doesn't have any column names. 
# We need to remember the mapping from the index to the column names.

# Access values in the first_row variable
print('First Row Values')
print(first_row)

In [ ]:
# SQL query.
sql_select = 'SELECT * FROM raw.inmt4aa1 LIMIT 10;'
pgsql_cursor.execute(sql_select)
result_list = pgsql_cursor.fetchmany(size=10)

print(type(result_list))
print(type(result_list[0]))

# loop
result_counter = 0
for i, result_row in enumerate(result_list):
    print('\n------------------------------------------------\n')
    print('Row {0}\n'.format(i))
    pp.pprint(result_row)

# Manipulating Tables

Please enter your andrew id here. We don't want to create the same table over and over again. 

In [ ]:
andrew_id = 'kit'
table_name = '{}_hobbies'.format(andrew_id)

Execute an appropriate CREATE statement on the cursor. Now go to DBeaver. Do you see the new table?

In [ ]:
create_table = '''CREATE TABLE IF NOT EXISTS scratch.{} (
    hobby varchar NULL,
    hours_per_week int
);'''.format(table_name)
pgsql_cursor.execute(create_table)

Statements that modify the state of the database will not be physically reflected until we tell the connection to commit these changes. 

In [ ]:
pgsql_connection.commit()

We can now insert records into the table and commit the changes.

In [ ]:
insert_into_table = '''INSERT INTO scratch.{}
    (hobby, hours_per_week)
    VALUES(%s, %s);
'''.format(table_name)
records_to_insert = [('improv', 10), ('sleep', 40)]
for record in records_to_insert:
    pgsql_cursor.execute(insert_into_table, record)
pgsql_connection.commit()

Finally, we will drop the table (and commit!)

In [ ]:
drop_table = 'DROP TABLE scratch.{};'.format(table_name)
pgsql_cursor.execute(drop_table)
pgsql_connection.commit()

# Data Visualization

Let's get the histogram of number of disciplinary infraction charges by inmate.

In [ ]:
sql_query = '''
        SELECT inmate_doc_id, COUNT(*) AS num_charges FROM raw.inmt4aa1 as inmates
        LEFT JOIN raw.INMT9CF1 as infraction
        ON inmates.inmate_doc_id = infraction.offender_doc_id
        GROUP BY inmate_doc_id;
'''

pgsql_cursor_dict.execute(sql_query)
num_charges_results = pgsql_cursor_dict.fetchall()

In [ ]:
print(len(num_charges_results))
for row in num_charges_results[:10]:
    pp.pprint({k: v for k, v in row.items()})

In [ ]:
plt.hist([row['num_charges'] for row in num_charges_results], bins=20)
plt.xlabel('Number of offenses')
plt.ylabel('Number of inmates')
plt.title('Histogram of number of disciplinary infraction charges by inmate')
plt.show()

In [ ]:
plt.hist(np.log([row['num_charges'] for row in num_charges_results]), bins=20)
plt.xlabel('Number of offenses (ln)')
plt.ylabel('Number of inmates')
plt.title('Histogram of number of disciplinary infraction charges by inmate')
plt.show()

Now let's try to join that information with some ethnic information

In [ ]:
sql_query = '''
    SELECT raw_inmates.inmate_doc_id, num_charges, inmate_ethnic 
    FROM
            (SELECT inmate_doc_id, COUNT(*) AS num_charges FROM raw.inmt4aa1 as inmates
            LEFT JOIN raw.INMT9CF1 as infraction
            ON inmates.inmate_doc_id = infraction.offender_doc_id
            GROUP BY inmate_doc_id) AS charges_table 
        JOIN raw.inmt4aa1 as raw_inmates ON raw_inmates.inmate_doc_id = charges_table.inmate_doc_id;
'''

pgsql_cursor_dict.execute(sql_query)
num_charges_results = pgsql_cursor_dict.fetchall()

In [ ]:
print(len(num_charges_results))
for row in num_charges_results[:10]:
    pp.pprint({k: v for k, v in row.items()})

In [ ]:
# prepare data to be plotted
data_by_ethnicity = {}
for row in num_charges_results:
    ethnicity = row['inmate_ethnic']
    if ethnicity not in data_by_ethnicity:
        data_by_ethnicity[ethnicity] = []
    data_by_ethnicity[ethnicity].append(row['num_charges'])

In [ ]:
for e in data_by_ethnicity:
    print(e, len(data_by_ethnicity[e]))

In [ ]:
to_plot = random.sample(list(data_by_ethnicity.keys()), 3)
data_to_plot = [data_by_ethnicity[e] for e in data_by_ethnicity if e in to_plot]
fig = plt.figure(1, figsize=(8, 5))
ax = fig.add_subplot(111)
bp = ax.boxplot(data_to_plot, sym='')
ax.set_xticklabels(to_plot)
plt.ylabel('num offenses')
plt.show()